In [1]:
import torch
from torch.utils.data import DataLoader
from state_quantization.dataset import load_dataset
from state_quantization.dataset import DynamicsModelDataset
from state_quantization.forcasting_models import LSTMForcasting
from state_quantization.quantization_models import DiscAutoEncoder
from state_quantization.forcasting_quantization_models import ForcastingQuant
from state_quantization.trainer import ForcastingQuantTrainer
from torch.optim.lr_scheduler import MultiStepLR
from state_quantization.train import train_model, test_step
from state_quantization.eval import eval_model, compare_models

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True

Using Device: cuda:0


## Load Dataset

In [3]:
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples-la.npy'
normalized_data_params_save_path = 'state_quantization/NormalizeInputConfigs.pkl'
dataset_device = 'cpu'
y_indexes = [4, 6]

train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key,
                                          output_key=dataset_output_key, dataset_class=DynamicsModelDataset,
                                          normalize=True, device=dataset_device, y_clip_range=y_indexes,
                                          normalized_data_params_save_path=normalized_data_params_save_path)

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last': True,
          'pin_memory': not train_dataset.x.is_cuda}
print(params)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)
train_dataset.y[0]

Clipping y
torch.Size([672000, 39, 6])
torch.Size([672000, 10, 2])
torch.Size([288000, 39, 6])
torch.Size([288000, 10, 2])
{'batch_size': 8000, 'shuffle': True, 'num_workers': 0, 'drop_last': True, 'pin_memory': True}


tensor([[ 1.6728, -0.8214],
        [ 1.6070, -0.8535],
        [ 1.7155, -0.8366],
        [ 1.7236, -0.8397],
        [ 1.7005, -0.8373],
        [ 1.6969, -0.8373],
        [ 1.8305, -0.8712],
        [ 1.7238, -0.8689],
        [ 1.8825, -0.8338],
        [ 1.7376, -0.8298]])

## Define model

In [4]:
model_path = 'state_quantization/model'

num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 20
out_size = train_dataset.get_output_feature_size()
print(f'Out Size:{out_size}')
look_ahead = train_dataset.get_look_ahead_size()
n_layers = 1
dropout = 0.1

forcasting_model = LSTMForcasting(features=num_of_features, hidden_size=hidden_size, out_size=out_size, seq_len=seq_len,
                                  look_ahead=look_ahead, dropout=dropout, n_layers=n_layers)

disc_autoencoder_input_size = hidden_size * 2
bottleneck_size = 20
ae_dropout = 0.0
disc_autoencoder = DiscAutoEncoder(input_size=disc_autoencoder_input_size, bottleneck_size=bottleneck_size,
                                   dropout=ae_dropout)

model = ForcastingQuant(forcasting_model=forcasting_model, autoencoder_quant_model=disc_autoencoder).to(device=device)

load_to_gpu = model.is_cuda() and not train_dataset.x.is_cuda
print(load_to_gpu)



Out Size:2
LSTM Layers
ModuleList(
  (0): LSTMCell(6, 20)
)
LSTM Dropout Layers
ModuleList()
Fully Connected Layers
Sequential(
  (0): Linear(in_features=20, out_features=20, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=20, out_features=20, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=20, out_features=2, bias=True)
)
[40, 20, 10]
[10, 20, 40]
Encoder Layers
Sequential(
  (0): Linear(in_features=40, out_features=40, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=40, out_features=20, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.0, inplace=False)
  (6): Linear(in_features=20, out_features=10, bias=True)
  (7): GELU(approximate=none)
  (8): Dropout(p=0.0, inplace=False)
)
Bottleneck Layers
Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): Tanh()
  (2): StraightThroughEstimator

## Define Trainer

In [5]:
gamma = 0.1

forecasting_learning_rate = 1e-3
autoencoder_learning_rate = 1e-3

forecasting_lr_milestones = [35]
autoencoder_lr_milestones = [100, 150]
forecasting_optimizer = torch.optim.Adam(model.forcasting_model.parameters(),
                                         lr=forecasting_learning_rate)
autoencoder_optimizer = torch.optim.Adam(model.autoencoder_quant_model.parameters(),
                                         lr=autoencoder_learning_rate)
forecasting_lr_scheduler = MultiStepLR(forecasting_optimizer, milestones=forecasting_lr_milestones, gamma=gamma)
autoencoder_lr_scheduler = MultiStepLR(autoencoder_optimizer, milestones=autoencoder_lr_milestones, gamma=gamma)
n_epochs = 200

trainer = ForcastingQuantTrainer(forcasting_quant_model=model, train_loader=train_loader, test_loader=val_loader,
                                 load_to_gpu=load_to_gpu, forecasting_optimizer=forecasting_optimizer,
                                 forecasting_lr_scheduler=forecasting_lr_scheduler,
                                 autoencoder_lr_scheduler=autoencoder_lr_scheduler,
                                 autoencoder_optimizer=autoencoder_optimizer,autoencoder_training_start=50)



## Train Model

In [6]:
trainer.train(n_epochs=n_epochs)

Untrained test
--------
--------------------------------------
Forcasting Test loss: 1.0063083602322473
--------------------------------------
--------------------------------------
Epoch 1
---------
--------------------------------------
Forcasting Train loss: 0.7652150293191274
--------------------------------------
Forcasting Test loss: 0.39165402121014065
--------------------------------------
Forecasting lr: [0.001]
Autoencoder lr: [0.001]
Epoch time: epoch_time = 9.020s
--------------------------------------
--------------------------------------
Epoch 2
---------


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


--------------------------------------
Forcasting Train loss: 0.22361169276492937
--------------------------------------
Forcasting Test loss: 0.09681832873158985
--------------------------------------
Forecasting lr: [0.001]
Autoencoder lr: [0.001]
Epoch time: epoch_time = 9.090s
--------------------------------------
--------------------------------------
Epoch 3
---------
--------------------------------------
Forcasting Train loss: 0.11247902168404489
--------------------------------------
Forcasting Test loss: 0.07131622524725066
--------------------------------------
Forecasting lr: [0.001]
Autoencoder lr: [0.001]
Epoch time: epoch_time = 8.814s
--------------------------------------
--------------------------------------
Epoch 4
---------
--------------------------------------
Forcasting Train loss: 0.09254223729173343
--------------------------------------
Forcasting Test loss: 0.05791924003925589
--------------------------------------
Forecasting lr: [0.001]
Autoencoder lr: [0

KeyboardInterrupt: 

## Save Model

In [7]:
torch.save(model, model_path)
torch.cuda.empty_cache()
del model

In [7]:
model_path = 'state_quantization/model'

model_dict_path = 'state_quantization/model_dict.pt'
device = 'cpu'
model = torch.load(model_path).to(device)
print(train_dataset.x[0:2].shape)
torch.onnx.export(model, train_dataset.x[0:2].to(device), 'state_quantization/model.onnx')

torch.Size([2, 39, 6])


RuntimeError: ONNX export failed: Couldn't export Python operator STEFunction

Defined at:
/home/hamza/PycharmProjects/StateCompression/state_quantization/activations.py(21): forward
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/module.py(1118): _slow_forward
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/module.py(1130): _call_impl
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/container.py(139): forward
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/module.py(1118): _slow_forward
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/module.py(1130): _call_impl
/home/hamza/PycharmProjects/StateCompression/state_quantization/quantization_models.py(201): forward
/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/torch/nn/modules/m